In [9]:
from google.colab import drive, files
import pandas as pd
import os

drive.mount('/content/drive')

def load_data_from_folder(folder_path, activity_label):
    data_list = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path)
            df['activity'] = activity_label
            data_list.append(df)
    return pd.concat(data_list, ignore_index=True)

data_path = '/content/drive/MyDrive/data'
folders = ['idle', 'running', 'stairs', 'walking']
labels = ['idle', 'running', 'stairs', 'walking']

data = pd.concat([load_data_from_folder(os.path.join(data_path, folder), label) for folder, label in zip(folders, labels)], ignore_index=True)

print(data.head())

data.to_csv('/content/formed_dataset.csv', index=False)

files.download('/content/formed_dataset.csv')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   accelerometer_X  accelerometer_Y  accelerometer_Z activity
0        -0.081403         0.196325         9.782708     idle
1        -0.124498         0.225055         9.768343     idle
2        -0.100556         0.201113         9.763555     idle
3        -0.124498         0.215478         9.773131     idle
4        -0.119710         0.220267         9.744401     idle
Index(['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z', 'activity'], dtype='object')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

data = pd.read_csv('/content/formed_dataset.csv')

print(data.columns)

window_size = 50

for axis in ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']:
    data[f'mean_{axis}'] = data[axis].rolling(window=window_size).mean()
    data[f'std_{axis}'] = data[axis].rolling(window=window_size).std()
    data[f'max_{axis}'] = data[axis].rolling(window=window_size).max()
    data[f'min_{axis}'] = data[axis].rolling(window=window_size).min()

data.dropna(inplace=True)

features = [f'mean_{axis}' for axis in ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']] + \
           [f'std_{axis}' for axis in ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']] + \
           [f'max_{axis}' for axis in ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']] + \
           [f'min_{axis}' for axis in ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']]
X = data[features]
y = data['activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm_model = SVC()
svm_model.fit(X_train_scaled, y_train)
y_pred_svm = svm_model.predict(X_test_scaled)

print("Classification Report for SVM:")
print(classification_report(y_test, y_pred_svm))

rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)

print("Classification Report for Random Forest:")
print(classification_report(y_test, y_pred_rf))


Index(['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z', 'activity'], dtype='object')
Classification Report for SVM:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      9291
     running       1.00      1.00      1.00     30609
      stairs       0.97      0.93      0.95      1500
     walking       0.99      1.00      1.00     16744

    accuracy                           1.00     58144
   macro avg       0.99      0.98      0.99     58144
weighted avg       1.00      1.00      1.00     58144

Classification Report for Random Forest:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      9291
     running       1.00      1.00      1.00     30609
      stairs       1.00      1.00      1.00      1500
     walking       1.00      1.00      1.00     16744

    accuracy                           1.00     58144
   macro avg       1.00      1.00      1.00     58144
weighted avg       1